In [1]:
import pandas as pd
import numpy as np
import tqdm
import seaborn as sns
from scipy import stats
from scipy.stats import f_oneway
from scipy.stats import chi2_contingency
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("online_shoppers_intention (1).csv")

In [4]:
df.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates in %,ExitRates in %,PageValues,SpecialDay (probability),Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,1,1,1,1,Returning_Visitor,False,False
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,Feb,2,2,1,2,Returning_Visitor,False,False
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,4,1,9,3,Returning_Visitor,False,False
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,Feb,3,2,2,4,Returning_Visitor,False,False
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,Feb,3,3,1,4,Returning_Visitor,True,False


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12330 entries, 0 to 12329
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Administrative            12330 non-null  int64  
 1   Administrative_Duration   12330 non-null  float64
 2   Informational             12330 non-null  int64  
 3   Informational_Duration    12330 non-null  float64
 4   ProductRelated            12330 non-null  int64  
 5   ProductRelated_Duration   12330 non-null  float64
 6   BounceRates in %          12330 non-null  float64
 7   ExitRates in %            12330 non-null  float64
 8   PageValues                12330 non-null  float64
 9   SpecialDay (probability)  12330 non-null  float64
 10  Month                     12330 non-null  object 
 11  OperatingSystems          12330 non-null  int64  
 12  Browser                   12330 non-null  int64  
 13  Region                    12330 non-null  int64  
 14  Traffi

In [3]:
cat_cols = [col for col in df.columns if df[col].dtypes == 'object']
bool_cols = [col for col in df.columns if df[col].dtypes == 'bool']
print(cat_cols)
print(bool_cols)

['Month', 'VisitorType']
['Weekend', 'Revenue']


In [25]:
pivot = df.pivot_table(index='VisitorType', columns='Revenue', aggfunc='size')
pivot

Revenue,False,True
VisitorType,,
New_Visitor,1272,422
Other,69,16
Returning_Visitor,9081,1470


In [27]:
chi2, p, dof, ex = chi2_contingency(pivot)
chi2, p, dof, ex

(135.2519228192047,
 4.269904152293867e-30,
 2,
 array([[1431.86277372,  262.13722628],
        [  71.84671533,   13.15328467],
        [8918.29051095, 1632.70948905]]))

In [30]:
pivot = df.pivot_table(index='Weekend', columns='Revenue', aggfunc='size')
pivot

Revenue,False,True
Weekend,,
False,8053,1409
True,2369,499


In [31]:
chi2, p, dof, ex = chi2_contingency(pivot)
chi2, p, dof, ex

(10.390978319534856,
 0.0012663251061221968,
 1,
 array([[7997.80729927, 1464.19270073],
        [2424.19270073,  443.80729927]]))

In [32]:
pivot = df.pivot_table(index='Month', columns='Revenue', aggfunc='size')
pivot

Revenue,False,True
Month,,
Aug,357,76
Dec,1511,216
Feb,181,3
Jul,366,66
June,259,29
Mar,1715,192
May,2999,365
Nov,2238,760
Oct,434,115


In [33]:
chi2, p, dof, ex = chi2_contingency(pivot)
chi2, p, dof, ex

(384.93476153599426,
 2.2387855164805443e-77,
 9,
 array([[ 365.99562044,   67.00437956],
        [1459.75620438,  267.24379562],
        [ 155.5270073 ,   28.4729927 ],
        [ 365.15036496,   66.84963504],
        [ 243.43357664,   44.56642336],
        [1611.90218978,  295.09781022],
        [2843.43941606,  520.56058394],
        [2534.07591241,  463.92408759],
        [ 464.04525547,   84.95474453],
        [ 378.67445255,   69.32554745]]))